# How to handle WeldX files
In this notebook we will demonstrate how to create, read, and update ASDF files created by WeldX. All the needed funcationality is contained in a single class named `WeldxFile`. We are going to show different modes of operation, like working with physical files on your harddrive, and in-memory files, both read-only and read-write mode.

In [ ]:
from weldx import WeldxFile
import numpy as np
import datetime

Now we create our first file, by invoking the `WeldxFile` constructor without any additional arguments. By doing so, we create an in-memory file. This means, that your changes will be temporary until you write it to an actual file on your harddrive. The `file_handle` attribute will point to the actual underlying file. In this case it is the in-memory file or buffer as shown below.

In [ ]:
file = WeldxFile()
file.file_handle

Next we assign some dictionary like data to the file, by storing it some attribute name enclosed by square brackets.
Then we look at the representation of the file header or contents. This will depend on the execution environment.
In JupyterLab you will see an interactive tree like structure, which can be expanded and searched.

In [ ]:
file["some_data"] = {"data_set": {"data": np.random.random(100), "time": "now"}}
file

We can use all dictionary operations on the data we like, e.g. update, assign, and delete items.

In [ ]:
file["some_data"]["another_data_set"] =  {"data": np.random.random(100), "time": "now"}

# delete the first data set again:

del file["some_data"]["data_set"]
file

We can now safe the memory-backed file to disk by invoking `WeldxFile.write_to`.

In [ ]:
file.write_to("example.asdf")

This newly created file can be opened up again, in read-write mode like by passing the appropriate arguments.

In [ ]:
example = WeldxFile("example.asdf", mode="rw")

TODO: context manager, special keywords to write, open etc.